In [1]:
import os
os.chdir('/home/xyz/data/haoyan/bandits')

import bandits as bd
import numpy as np
import pandas as pd

from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

/home/xyz/miniconda3/envs/bandit/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
data = pd.read_csv('/home/xyz/data/haoyan/bandits/data/Walmart_sales.csv')
data['Date'] = pd.to_datetime(data['Date'], dayfirst=True)
data['month'] = data['Date'].dt.month

x = data[['Store', 'month', 'Holiday_Flag', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment']]
y = data['Weekly_Sales']

x_train, x_test, y_train, y_test=train_test_split(x,y,train_size=0.67,random_state=34)

In [3]:
def g(
    max_bin,
    n_estimators,
    max_depth,
    colsample_bytree,
    learning_rate,
    reg_alpha,
    reg_lambda,
):
    config = {
        "max_bin": max_bin,
        "n_estimators": n_estimators,
        "max_depth": max_depth,
        "colsample_bytree": min(colsample_bytree, 1),
        "learning_rate": learning_rate,
        "reg_alpha": reg_alpha,
        "reg_lambda": reg_lambda,
        "random_state": 34,
        "tree_method": "hist",
    }

    model = XGBRegressor(**config)
    model.fit(x_train, y_train, verbose=1)
    predictions = model.predict(x_test)
    return -mean_absolute_error(y_test, predictions) / 1e5

In [4]:
n_experiment = 10

# gradient bandit hybrid

In [ ]:
params_list_dict = {
    "max_bin": [2**6, 2**7, 2**8, 2**9],
    "max_depth": [4, 5, 6, 7, 8, 9],
    "n_estimators": [500, 1000, 2500, 5000],
}

pbounds={
    "colsample_bytree": [0.2, 1],
    'learning_rate': [0.001, 0.5],
    'reg_alpha': [0, 3],
    'reg_lambda': [0, 3],
}

scores_list = []
actions_list = []

def single_func(n, f):
    bandit = bd.GaussianBandit(
        g,
        params_list_dict=params_list_dict,
        pbounds=pbounds,
        random_state=n*128,
        init_points=5,
        n_iter=5,
    )

    policy = bd.SoftmaxPolicy()
    agent = bd.GradientAgent(bandit, policy, alpha=1.5, avg_coef=None, baseline=True, increase_rate=0)
    env = bd.Environment(bandit, agent, 'Gradient Agents', random_state=n*128)
    scores, actions = env.run(600, file=f)
    
    return scores, actions

if os.path.exists('/home/xyz/data/haoyan/bandits/logs/rl_hybrid_results.txt'):
    os.remove('/home/xyz/data/haoyan/bandits/logs/rl_hybrid_results.txt')

with open('/home/xyz/data/haoyan/bandits/logs/rl_hybrid_results.txt', 'a') as f:
    res = [single_func(x, f) for x in range(n_experiment)]

0
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 1         | -0.6467   | 0.4823    | 0.2561    | 0.299     | 2.68      |
| 2         | -0.6533   | 0.5632    | 0.1782    | 1.014     | 2.144     |
| 3         | -0.6025   | 0.8644    | 0.06601   | 2.674     | 1.117     |
| 4         | -0.6432   | 0.4645    | 0.2735    | 0.9338    | 1.485     |
| 5         | -0.6712   | 0.5399    | 0.295     | 0.04765   | 1.223     |
1
((256, 5, 500), {'colsample_bytree': 0.8643874208698772, 'learning_rate': 0.06600936010158626, 'reg_alpha': 2.6735991126556025, 'reg_lambda': 1.116634959901813}) -0.6024817115672081
1
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 1         | -0.652    | 0.6908    | 0.1431    | 2.781     | 1.376     |
| 2         | -0.8745   | 0.3362    | 0.406     | 2.483     | 2.997     

| 1         | -0.6545   | 0.9548    | 0.2691    | 2.464     | 2.636     |
| 2         | -0.6383   | 0.6613    | 0.2465    | 2.046     | 1.71      |
| 3         | -0.6919   | 0.5492    | 0.3972    | 0.5367    | 1.273     |
| 4         | -0.694    | 0.6164    | 0.4721    | 0.3377    | 2.861     |
| 5         | -1.22     | 0.7601    | 0.004449  | 1.478     | 0.6953    |
1.0
((512, 9, 500), {'colsample_bytree': 0.6612792558422771, 'learning_rate': 0.24651157276195934, 'reg_alpha': 2.046058317149533, 'reg_lambda': 1.709918604290812}) -0.6382639879914077
9
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 1         | -0.6599   | 0.4473    | 0.1262    | 2.955     | 1.759     |
| 2         | -0.6922   | 0.8382    | 0.4064    | 1.428     | 1.025     |
| 3         | -0.8791   | 0.2663    | 0.2246    | 1.252     | 1.557     |
| 4         | -0.6077   | 0.9183    | 0.08382   | 1.482     | 1.68      |

| 1         | -0.6615   | 0.6139    | 0.1652    | 2.516     | 0.2738    |
| 2         | -0.6341   | 0.6302    | 0.1294    | 1.678     | 2.599     |
| 3         | -0.6209   | 0.8021    | 0.06955   | 2.861     | 2.194     |
| 4         | -0.693    | 0.7992    | 0.4738    | 2.349     | 2.354     |
| 5         | -0.6259   | 0.8215    | 0.05415   | 0.2614    | 1.651     |
1.0
((64, 4, 1000), {'colsample_bytree': 0.8020591096582128, 'learning_rate': 0.06954918092016876, 'reg_alpha': 2.8610735624408474, 'reg_lambda': 2.194321697972442}) -0.620916886080582
17
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 1         | -0.9474   | 0.2034    | 0.4522    | 2.197     | 1.817     |
| 2         | -0.608    | 0.717     | 0.1176    | 1.734     | 0.5162    |
| 3         | -0.9518   | 0.2424    | 0.4662    | 0.395     | 0.7103    |
| 4         | -0.7586   | 0.3087    | 0.3076    | 0.1774    | 0.8003    

| 1         | -0.714    | 0.9924    | 0.463     | 1.96      | 0.9749    |
| 2         | -0.6683   | 0.6686    | 0.1123    | 0.1386    | 1.57      |
| 3         | -0.6556   | 0.9048    | 0.2538    | 2.158     | 2.483     |
| 4         | -0.6561   | 0.881     | 0.1454    | 0.2832    | 0.653     |
| 5         | -0.7061   | 0.945     | 0.4536    | 0.8071    | 1.964     |
1.0
((64, 6, 5000), {'colsample_bytree': 0.9047906883053052, 'learning_rate': 0.2537657487892374, 'reg_alpha': 2.158437569425555, 'reg_lambda': 2.4830320370137935}) -0.655604096854402
25
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 1         | -0.6456   | 0.6936    | 0.3026    | 1.027     | 2.638     |
| 2         | -0.6927   | 0.3475    | 0.05513   | 2.712     | 2.896     |
| 3         | -0.6844   | 0.9972    | 0.3391    | 1.226     | 0.3799    |
| 4         | -0.6914   | 0.5061    | 0.4267    | 1.821     | 2.038     |

| 1         | -0.6594   | 0.9813    | 0.2643    | 0.3237    | 0.4791    |
| 2         | -0.8894   | 0.211     | 0.1981    | 2.159     | 1.658     |
| 3         | -0.7013   | 0.5552    | 0.1808    | 1.636     | 1.756     |
| 4         | -0.658    | 0.8988    | 0.1038    | 0.7032    | 0.05557   |
| 5         | -0.8988   | 0.3941    | 0.3392    | 2.155     | 1.865     |
1.0
((128, 8, 5000), {'colsample_bytree': 0.8987601065828761, 'learning_rate': 0.10375345512205866, 'reg_alpha': 0.703189622240564, 'reg_lambda': 0.05557111227363787}) -0.6580072539283193
33
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 6         | -0.6743   | 0.5451    | 0.1726    | 1.286     | 1.533     |
| 7         | -0.6396   | 0.6853    | 0.2669    | 0.3265    | 2.539     |
| 8         | -0.6413   | 0.9068    | 0.3751    | 1.358     | 0.9826    |
| 9         | -0.6703   | 0.7874    | 0.4015    | 1.373     | 1.773  

...

| 21        | -0.8947   | 0.2       | 0.5       | 0.0       | 2.328     |
| 22        | -0.9095   | 0.2       | 0.5       | 1.186     | 0.0       |
| 23        | -4.375    | 0.2       | 0.001     | 0.0       | 3.0       |
| 24        | -0.6337   | 1.0       | 0.01428   | 2.513     | 1.046     |
| 25        | -0.7421   | 0.5206    | 0.5       | 2.524     | 0.9164    |
1.0
((512, 9, 500), {'colsample_bytree': 1.0, 'learning_rate': 0.014279177234917248, 'reg_alpha': 2.5127766624887884, 'reg_lambda': 1.0460466668437556}) -0.6336829870350754
300
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 6         | -0.6521   | 0.6216    | 0.4149    | 1.737     | 1.906     |
| 7         | -0.5932   | 0.9233    | 0.07288   | 1.851     | 2.133     |
| 8         | -0.6171   | 1.0       | 0.3631    | 1.727     | 2.269     |
| 9         | -0.6206   | 0.964     | 0.2388    | 2.598     | 1.011     |
| 10     

| 17        | -0.6521   | 1.0       | 0.3601    | 1.62      | 2.08      |
| 18        | -0.7251   | 0.6607    | 0.5       | 1.48      | 1.864     |
| 19        | -0.7385   | 0.5715    | 0.4704    | 0.9597    | 0.8478    |
| 20        | -0.6975   | 0.9331    | 0.5       | 1.828     | 1.574     |
1.0
((64, 5, 1000), {'colsample_bytree': 0.8207228522185104, 'learning_rate': 0.07901109564941547, 'reg_alpha': 1.4990371601269725, 'reg_lambda': 2.3846707193426058}) -0.6180761818201213
308
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 21        | -0.6527   | 0.9011    | 0.09982   | 2.437     | 0.357     |
| 22        | -0.6635   | 0.9369    | 0.07364   | 2.635     | 0.02711   |
| 23        | -0.9118   | 0.2457    | 0.2788    | 2.009     | 1.658     |
| 24        | -0.9121   | 0.259     | 0.3427    | 0.1292    | 0.4643    |
| 25        | -0.6895   | 0.7674    | 0.31      | 0.6423    | 0.4667 

| 13        | -0.69     | 1.0       | 0.5       | 0.9692    | 1.027     |
| 14        | -4.577    | 0.2       | 0.001     | 2.211     | 1.356     |
| 15        | -0.676    | 0.7169    | 0.3424    | 1.608     | 0.8642    |
1.0
((128, 4, 500), {'colsample_bytree': 0.8146887873107378, 'learning_rate': 0.3007953284591458, 'reg_alpha': 2.255832088626962, 'reg_lambda': 1.073637405059964}) -0.6384815040857462
316
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 21        | -0.601    | 0.9823    | 0.01408   | 2.177     | 2.646     |
| 22        | -0.6297   | 1.0       | 0.3359    | 2.125     | 2.615     |
| 23        | -0.6151   | 1.0       | 0.03701   | 1.923     | 2.317     |
| 24        | -2.841    | 0.6512    | 0.001     | 1.982     | 2.447     |
| 25        | -0.6384   | 1.0       | 0.3783    | 1.678     | 2.56      |
1.0
((128, 8, 1000), {'colsample_bytree': 0.9823246874361102, 'learning_

| 29        | -0.6961   | 1.0       | 0.5       | 2.214     | 0.8529    |
| 30        | -1.595    | 1.0       | 0.001     | 1.224     | 1.407     |
1.0
((128, 5, 2500), {'colsample_bytree': 0.5962057978672279, 'learning_rate': 0.034242777039448904, 'reg_alpha': 2.435913828218915, 'reg_lambda': 2.8554610039881894}) -0.6261422193900071
324
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 11        | -4.02     | 0.2       | 0.001     | 2.406     | 1.622     |
| 12        | -0.7868   | 0.3301    | 0.5       | 2.1       | 2.736     |
| 13        | -0.6909   | 1.0       | 0.5       | 2.986     | 2.091     |
| 14        | -0.8869   | 0.2078    | 0.3598    | 2.719     | 1.487     |
| 15        | -0.7453   | 0.5623    | 0.5       | 3.0       | 1.699     |
1.0
((128, 4, 2500), {'colsample_bytree': 0.9714694654038472, 'learning_rate': 0.027232622453977434, 'reg_alpha': 2.382791651893558, 'reg_lamb

| 25        | -0.8581   | 0.2       | 0.5       | 1.566     | 0.192     |
1.0
((512, 5, 500), {'colsample_bytree': 0.7196560339098574, 'learning_rate': 0.18312092231522512, 'reg_alpha': 0.0760927261084815, 'reg_lambda': 2.967358875019803}) -0.597971961600312
332
|   iter    |  target   | colsam... | learni... | reg_alpha | reg_la... |
-------------------------------------------------------------------------
| 26        | -0.6338   | 1.0       | 0.1705    | 1.621     | 1.368     |
| 27        | -0.676    | 0.991     | 0.3583    | 1.654     | 2.044     |
| 28        | -0.6786   | 0.8583    | 0.423     | 2.297     | 2.135     |
| 29        | -0.7248   | 0.8205    | 0.4826    | 2.286     | 1.337     |
| 30        | -0.6625   | 0.5503    | 0.1351    | 1.408     | 1.397     |
1.0
((64, 5, 1000), {'colsample_bytree': 0.8207228522185104, 'learning_rate': 0.07901109564941547, 'reg_alpha': 1.4990371601269725, 'reg_lambda': 2.3846707193426058}) -0.6180761818201213
333
|   iter    |  target   | co

In [6]:
scores_list = []
actions_list = []
for x in res:
    scores_list.append(x[0])
    actions_list.append(x[1])

NameError: name 'res' is not defined

In [ ]:
for scores, actions in zip(scores_list, actions_list):
    print(actions[np.argmax(scores)], scores[np.argmax(scores)], len(actions))

# gradient bandit random search

In [7]:
params_list_dict = {
    "max_bin": [2**4, 2**5, 2**6, 2**7, 2**8, 2**9, 2**10],
    "max_depth": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    "n_estimators": [100, 500, 1000, 2500, 5000],
}

pbounds={
    "colsample_bytree": [0.2, 1],
    'learning_rate': [0.001, 0.5],
    'reg_alpha': [0, 3],
    'reg_lambda': [0, 3],
}

scores_list = []
actions_list = []

def single_func(n, f):
    bandit = bd.GaussianBanditRandomSearch(
        g,
        params_list_dict=params_list_dict,
        pbounds=pbounds,
        random_state=n*128,
        max_single_discrete_param_n_iter=300,
    )

    policy = bd.SoftmaxPolicy()
    agent = bd.GradientAgent(bandit, policy, alpha=0.5, avg_coef=None, baseline=True, increase_rate=0)
    env = bd.Environment(bandit, agent, 'Gradient Agents', random_state=n*128)
    scores, actions = env.run(600, file=f)
    
    return scores, actions

if os.path.exists('/home/xyz/data/haoyan/bandits/logs/rl_random_search_results.txt'):
    os.remove('/home/xyz/data/haoyan/bandits/logs/rl_random_search_results.txt')

with open('/home/xyz/data/haoyan/bandits/logs/rl_random_search_results.txt', 'a') as f:
    res = [single_func(x, f) for x in range(n_experiment)]

0
0.01
((256, 3, 5000), ({'colsample_bytree': 0.9088623519724577, 'learning_rate': 0.09252775479126046, 'reg_alpha': 2.661254733388616, 'reg_lambda': 0.16962686687613882}, {'max_bin': 256, 'max_depth': 3, 'n_estimators': 5000})) -0.6564767188301849
1
0.01
((16, 4, 5000), ({'colsample_bytree': 0.47552555662018686, 'learning_rate': 0.036370488973076986, 'reg_alpha': 1.1698756365655822, 'reg_lambda': 1.14154324892839}, {'max_bin': 16, 'max_depth': 4, 'n_estimators': 5000})) -1.1774656444275247
2
0.01
((64, 4, 100), ({'colsample_bytree': 0.7044285708342211, 'learning_rate': 0.0931157957153011, 'reg_alpha': 1.665212254259142, 'reg_lambda': 1.0992101779687005}, {'max_bin': 64, 'max_depth': 4, 'n_estimators': 100})) -1.115862739277307
3


KeyboardInterrupt: 

# random search

In [8]:
import time
import numpy as np
import scipy.stats as stats

from sklearn.model_selection import ParameterSampler


# specify parameters and distributions to sample from
params_list_dict = {
    "max_bin": [2**4, 2**5, 2**6, 2**7, 2**8, 2**9, 2**10],
    "n_estimators": [100, 1000, 2000, 5000],
    "max_depth": [2, 3, 4, 5, 6],
    "colsample_bytree": stats.uniform(0.2, 1),
    'learning_rate': stats.uniform(0.001, 0.5),
    'reg_alpha': stats.uniform(0, 3),
    'reg_lambda': stats.uniform(0, 3),
}

# n_iter = max([len(actions) for actions in actions_list])
n_iter = 3000
n_experiment = 10

res_dict_list = []

if os.path.exists('/home/xyz/data/haoyan/bandits/logs/grid_search_results.txt'):
    os.remove('/home/xyz/data/haoyan/bandits/logs/grid_search_results.txt')

with open('/home/xyz/data/haoyan/bandits/logs/grid_search_results.txt', 'a') as f:
    for n in range(n_experiment):
        param_list = list(ParameterSampler(params_list_dict, n_iter=n_iter, random_state=n*128))
        res_dict = {}
        for i, param in enumerate(param_list):
            start = time.time()
            res_dict[str(param)] = g(**param)
            end = time.time()
            f.write(f'({n*128}, {i}, {param}, {res_dict[str(param)]}, {end-start})\n')
            if i % 10 == 0:
                print(i, param, res_dict[str(param)])
            
        res_dict_list.append(res_dict)

In [ ]:
for res_dict in res_dict_list:
    inverse = [(value, key) for key, value in res_dict.items()]
    print(max(inverse))

# bayesian opt

In [11]:
from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events

def g_hat(max_bin_exp,
        n_estimators,
        max_depth,
        colsample_bytree,
        learning_rate,
        reg_alpha,
        reg_lambda,):
    estimator_list = np.array([100, 1000, 2000, 5000])
    tmp = abs(estimator_list - n_estimators)
    n_estimators = estimator_list[np.argmin(tmp)]
    return g(
        2**int(max_bin_exp),
        n_estimators,
        int(max_depth),
        colsample_bytree,
        learning_rate,
        reg_alpha,
        reg_lambda,
    )

pbounds = {
    'max_bin_exp': (4, 10),
    'n_estimators': (100, 5000),
    "max_depth": (1, 6),
    "colsample_bytree": (0.2, 1),
    'learning_rate': (0.001, 0.5),
    'reg_alpha': (0, 3),
    'reg_lambda': (0, 3),
}

b_res_list = []
for n in range(n_experiment):
    optimizer = BayesianOptimization(
        f=g_hat,
        pbounds=pbounds,
        random_state=n*128,
        allow_duplicate_points=False
    )
    logger = JSONLogger(path=f"/home/xyz/data/haoyan/bandits/logs/bayesian_results_{n*128}.log")
    optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

    optimizer.maximize(init_points=3, n_iter=597)
    b_res_list.append(optimizer.max)

In [ ]:
[(x['target'], str({'u': int(int(x['params']['u'])/n_interval_u) * n_interval_u + 1, 'v': int(int(x['params']['v'])/n_interval_v) * n_interval_v + 1, 'w': x['params']['w'], 'x': int(int(x['params']['x'])/n_interval_x) * n_interval_x + 1, 'y': x['params']['y'], 'z': x['params']['z']})) for x in b_res_list]

[(0.2768580338893678,
  "{'u': 4, 'v': 7, 'w': 5.116789806249014, 'x': 13, 'y': 10.080905428791336, 'z': 0.9911011711627078}"),
 (0.1402183484847389,
  "{'u': 4, 'v': 7, 'w': 5.011204786829208, 'x': 1, 'y': 10.000305831311975, 'z': 0.9868788787576984}"),
 (0.27722306853469403,
  "{'u': 4, 'v': 7, 'w': 4.902417985793109, 'x': 13, 'y': 9.945820186243077, 'z': 0.924097238551662}"),
 (0.2768119314094586,
  "{'u': 4, 'v': 7, 'w': 4.885291040699603, 'x': 13, 'y': 10.081867600308184, 'z': 1.0257453084676125}"),
 (0.2768976627971634,
  "{'u': 4, 'v': 7, 'w': 5.09077481582764, 'x': 13, 'y': 9.937800034531513, 'z': 0.9109290708881388}"),
 (0.2799674220581756,
  "{'u': 4, 'v': 7, 'w': 4.948148565613696, 'x': 13, 'y': 10.012700410959875, 'z': 0.9925553408392699}"),
 (0.2753907393071401,
  "{'u': 4, 'v': 7, 'w': 5.075218248621907, 'x': 13, 'y': 9.860434685868848, 'z': 0.9419097111446684}"),
 (0.13862511632972735,
  "{'u': 4, 'v': 7, 'w': 5.048247231101259, 'x': 1, 'y': 10.066125798110978, 'z': 0.89

# rl

In [ ]:
n_trials = 3000

params_list_dict = {
    "max_bin": [2**6, 2**7, 2**8, 2**9],
    "max_depth": [4, 5, 6, 7, 8, 9],
    "n_estimators": [500, 1000, 2500, 5000],
    "colsample_bytree": np.linspace(0.2, 1, 10),
    'learning_rate': np.linspace(0.001, 0.5, 30),
    'reg_alpha': np.linspace(0, 3, 30),
    'reg_lambda': np.linspace(0, 3, 30),
}

scores_list_ori = []
actions_list_ori = []

if os.path.exists('/home/xyz/data/haoyan/bandits/logs/rl_results.txt'):
    os.remove('/home/xyz/data/haoyan/bandits/logs/rl_results.txt')

with open('/home/xyz/data/haoyan/bandits/logs/rl_results.txt', 'a') as f:

    for n in range(n_experiment):
        bandit = bd.GaussianBanditOri(
            g,
            params_list_dict=params_list_dict,
        )

        policy = bd.SoftmaxPolicy()
        agent = bd.GradientAgent(bandit, policy, alpha=0.1, baseline=True, increase_rate=0)
        env = bd.Environment(bandit, agent, 'Gradient Agents', random_state=n*128)
        scores, actions = env.run(n_trials, file=f)

        idx = np.argmax(scores)

        scores_list_ori.append(scores)
        actions_list_ori.append(actions)

0
0.1
((1024, 2000, 6, 0.9111111111111112, 0.3279310344827586, 2.2758620689655173, 2.0689655172413794), None) -0.6260685358179585
1
0.1
((1024, 100, 5, 0.37777777777777777, 0.3107241379310345, 0.20689655172413793, 1.5517241379310345), None) -0.8328349966322386
2
0.1
((64, 5000, 6, 0.2888888888888889, 0.15586206896551724, 0.41379310344827586, 2.0689655172413794), None) -0.8418240917834782
3
0.1
((128, 2000, 3, 0.2, 0.20748275862068966, 2.793103448275862, 2.2758620689655173), None) -0.8793157143802965
4
0.1
((256, 2000, 5, 0.37777777777777777, 0.19027586206896552, 2.4827586206896552, 1.9655172413793103), None) -0.7167884216563309
5
0.1
((256, 1000, 4, 0.7333333333333334, 0.2246896551724138, 2.793103448275862, 3.0), None) -0.6198023924045138
6
0.1
((32, 1000, 3, 0.8222222222222222, 0.4655862068965517, 0.5172413793103449, 0.5172413793103449), None) -0.7707729103842986
7
0.1
((128, 100, 2, 0.7333333333333334, 0.001, 1.7586206896551724, 1.4482758620689655), None) -4.656989500353108
8
0.1
((6

0.1
((16, 2000, 4, 0.2, 0.3623448275862069, 2.689655172413793, 0.3103448275862069), None) -3.1434687779822856
67
0.1
((16, 5000, 4, 0.7333333333333334, 0.17306896551724138, 0.8275862068965517, 2.1724137931034484), None) -1.0883461385813915
68
0.1
((128, 5000, 6, 0.2888888888888889, 0.4483793103448276, 1.8620689655172413, 2.689655172413793), None) -0.8829803433260414
69
0.1
((1024, 1000, 6, 0.5555555555555556, 0.24189655172413793, 2.4827586206896552, 1.4482758620689655), None) -0.6459754424243762
70
0.1
((256, 2000, 4, 0.9111111111111112, 0.06982758620689655, 2.1724137931034484, 0.10344827586206896), None) -0.6265455392383181
71
0.1
((16, 2000, 4, 0.2888888888888889, 0.4139655172413793, 0.5172413793103449, 0.0), None) -1.3235876037238112
72
0.1
((512, 1000, 5, 0.9111111111111112, 0.3967586206896552, 2.1724137931034484, 0.20689655172413793), None) -0.6601016519479754
73
0.1
((1024, 5000, 5, 0.8222222222222222, 0.08703448275862069, 0.20689655172413793, 2.586206896551724), None) -0.5977373

0.1
((1024, 1000, 5, 0.7333333333333334, 0.17306896551724138, 1.1379310344827587, 0.41379310344827586), None) -0.6130731549817561
134
0.1
((32, 100, 2, 0.2888888888888889, 0.2763103448275862, 0.3103448275862069, 2.2758620689655173), None) -2.2348664159545666
135
0.1
((64, 2000, 5, 0.4666666666666667, 0.19027586206896552, 3.0, 0.5172413793103449), None) -0.7251769841447446
136
0.1
((16, 1000, 6, 0.9111111111111112, 0.17306896551724138, 1.2413793103448276, 2.3793103448275863), None) -1.0351576604946444
137
0.1
((64, 100, 4, 0.7333333333333334, 0.3967586206896552, 0.6206896551724138, 2.793103448275862), None) -0.6688710296418904
138
0.1
((128, 100, 3, 1.0, 0.19027586206896552, 0.7241379310344828, 0.6206896551724138), None) -0.8355957175729757
139
0.1
((64, 100, 4, 0.2, 0.24189655172413793, 2.896551724137931, 1.4482758620689655), None) -1.6234752791696092
140
0.1
((256, 100, 6, 0.2888888888888889, 0.1386551724137931, 1.8620689655172413, 0.9310344827586207), None) -0.9838842335657955
141
0.

0.1
((64, 2000, 4, 0.2, 0.01820689655172414, 2.4827586206896552, 1.0344827586206897), None) -1.5049309910516715
200
0.1
((512, 2000, 4, 0.37777777777777777, 0.10424137931034483, 2.0689655172413794, 1.9655172413793103), None) -0.6658038511202422
201
0.1
((256, 1000, 2, 0.5555555555555556, 0.19027586206896552, 0.7241379310344828, 1.4482758620689655), None) -0.7442135368570504
202
0.1
((16, 5000, 6, 0.2888888888888889, 0.3107241379310345, 1.7586206896551724, 2.689655172413793), None) -1.4252912974035574
203
0.1
((256, 100, 2, 0.2, 0.3279310344827586, 2.896551724137931, 2.689655172413793), None) -2.5765958818032013
204
0.1
((16, 5000, 4, 0.8222222222222222, 0.43117241379310345, 0.41379310344827586, 2.0689655172413794), None) -1.1573290225671455
205
0.1
((16, 1000, 3, 0.9111111111111112, 0.12144827586206897, 1.1379310344827587, 1.5517241379310345), None) -1.2582422880303965
206
0.1
((128, 2000, 2, 0.6444444444444445, 0.3623448275862069, 2.4827586206896552, 0.8275862068965517), None) -0.7015

0.1
((32, 2000, 4, 1.0, 0.17306896551724138, 0.3103448275862069, 1.2413793103448276), None) -0.7497864386277366
266
0.1
((1024, 5000, 6, 0.2, 0.24189655172413793, 2.2758620689655173, 2.3793103448275863), None) -0.9694537991393009
267
0.1
((64, 2000, 2, 0.9111111111111112, 0.43117241379310345, 1.8620689655172413, 1.8620689655172413), None) -0.7056774155985168
268
0.1
((1024, 1000, 5, 0.2, 0.10424137931034483, 0.3103448275862069, 1.7586206896551724), None) -0.8573869944812265
269
0.1
((512, 1000, 3, 0.5555555555555556, 0.37955172413793103, 0.7241379310344828, 1.3448275862068966), None) -0.6789993165563207
270
0.1
((64, 2000, 3, 0.8222222222222222, 0.29351724137931035, 2.793103448275862, 0.6206896551724138), None) -0.6839185483779131
271
0.1
((16, 2000, 4, 0.2888888888888889, 0.4139655172413793, 2.4827586206896552, 1.4482758620689655), None) -1.3204220489222411
272
0.1
((64, 5000, 3, 0.8222222222222222, 0.4655862068965517, 2.896551724137931, 1.3448275862068966), None) -0.7640960120542019


...

0.1
((512, 1000, 3, 0.5555555555555556, 0.1386551724137931, 0.6206896551724138, 3.0), None) -0.6738574899603488
251
0.1
((32, 100, 5, 0.2888888888888889, 0.001, 0.41379310344827586, 2.793103448275862), None) -4.6605426274835215
252
0.1
((512, 100, 4, 0.2888888888888889, 0.3623448275862069, 0.6206896551724138, 1.0344827586206897), None) -0.8767266060469633
253
0.1
((32, 5000, 3, 1.0, 0.08703448275862069, 2.793103448275862, 0.20689655172413793), None) -0.7429487315674288
254
0.1
((16, 1000, 5, 0.8222222222222222, 0.19027586206896552, 1.2413793103448276, 2.689655172413793), None) -1.0517050458737935
255
0.1
((256, 100, 6, 0.2888888888888889, 0.12144827586206897, 1.6551724137931034, 2.896551724137931), None) -1.0276292603048494
256
0.1
((64, 100, 4, 1.0, 0.3623448275862069, 2.586206896551724, 1.9655172413793103), None) -0.6317370445386065
257
0.1
((1024, 100, 3, 0.8222222222222222, 0.25910344827586207, 2.0689655172413794, 2.1724137931034484), None) -0.8073495081479518
258
0.1
((32, 1000, 4

0.1
((256, 100, 3, 0.8222222222222222, 0.25910344827586207, 1.5517241379310345, 0.8275862068965517), None) -0.7963603194503296
318
0.1
((1024, 1000, 4, 0.37777777777777777, 0.24189655172413793, 1.2413793103448276, 2.689655172413793), None) -0.6846141769034692
319
0.1
((32, 100, 2, 0.37777777777777777, 0.5, 3.0, 2.2758620689655173), None) -1.6838373549111347
320
0.1
((128, 5000, 5, 0.5555555555555556, 0.20748275862068966, 0.8275862068965517, 1.8620689655172413), None) -0.6986671263933028
321
0.1
((256, 5000, 3, 1.0, 0.4483793103448276, 1.0344827586206897, 2.4827586206896552), None) -0.7200505861816177
322
0.1
((512, 1000, 5, 0.37777777777777777, 0.3107241379310345, 2.0689655172413794, 3.0), None) -0.699354004497475
323
0.1
((64, 2000, 2, 1.0, 0.4483793103448276, 2.2758620689655173, 0.5172413793103449), None) -0.7118250286885005
324
0.1
((32, 1000, 6, 0.37777777777777777, 0.2246896551724138, 1.7586206896551724, 1.9655172413793103), None) -0.8521338136556911
325
0.1
((128, 100, 3, 0.55555

0.1
((256, 5000, 2, 0.37777777777777777, 0.10424137931034483, 2.586206896551724, 2.1724137931034484), None) -0.7412676933380414
384
0.1
((512, 1000, 3, 1.0, 0.5, 0.8275862068965517, 2.2758620689655173), None) -0.6658537372116751
385
0.1
((256, 5000, 4, 0.5555555555555556, 0.29351724137931035, 0.0, 2.3793103448275863), None) -0.7114154139481079
386
0.1
((64, 100, 6, 0.37777777777777777, 0.24189655172413793, 0.0, 2.3793103448275863), None) -0.7835094865193915
387
0.1
((128, 100, 4, 0.6444444444444445, 0.37955172413793103, 0.3103448275862069, 0.7241379310344828), None) -0.6903573071195269
388
0.1
((256, 100, 2, 0.8222222222222222, 0.3279310344827586, 0.0, 2.0689655172413794), None) -1.0233757000706214
389
0.1
((64, 1000, 6, 0.9111111111111112, 0.19027586206896552, 2.793103448275862, 1.6551724137931034), None) -0.6376681099774895
390
0.1
((128, 100, 3, 1.0, 0.3107241379310345, 1.2413793103448276, 1.2413793103448276), None) -0.743894241682483
391
0.1
((256, 2000, 5, 0.37777777777777777, 0.5

0.1
((16, 2000, 4, 0.9111111111111112, 0.20748275862068966, 1.8620689655172413, 0.7241379310344828), None) -1.0870106469624035
451
0.1
((1024, 2000, 5, 0.8222222222222222, 0.15586206896551724, 2.4827586206896552, 1.0344827586206897), None) -0.6011991885637359
452
0.1
((512, 2000, 3, 0.6444444444444445, 0.052620689655172415, 2.3793103448275863, 2.1724137931034484), None) -0.6509203497108933
453
0.1
((16, 1000, 3, 0.5555555555555556, 0.2246896551724138, 1.6551724137931034, 0.5172413793103449), None) -1.2736633029013653
454
0.1
((16, 2000, 2, 0.8222222222222222, 0.08703448275862069, 0.9310344827586207, 0.20689655172413793), None) -1.7516956391550065
455
0.1
((32, 2000, 4, 0.37777777777777777, 0.48279310344827586, 1.4482758620689655, 2.0689655172413794), None) -0.8752010279756649
456
0.1
((512, 1000, 6, 0.5555555555555556, 0.29351724137931035, 0.3103448275862069, 2.586206896551724), None) -0.6575992973995114
457
0.1
((64, 1000, 3, 0.5555555555555556, 0.001, 1.7586206896551724, 0.0), None) 

0.1
((32, 2000, 2, 0.6444444444444445, 0.29351724137931035, 0.0, 2.4827586206896552), None) -0.8103826461143478
517
0.1
((128, 5000, 3, 0.6444444444444445, 0.43117241379310345, 1.2413793103448276, 0.10344827586206896), None) -0.7691932451575786
518
0.1
((128, 1000, 2, 0.6444444444444445, 0.24189655172413793, 0.9310344827586207, 3.0), None) -0.7239669985331333
519
0.1
((512, 5000, 2, 0.5555555555555556, 0.08703448275862069, 1.9655172413793103, 0.0), None) -0.7091701003759123
520
0.1
((1024, 2000, 6, 0.6444444444444445, 0.03541379310344828, 0.10344827586206896, 1.1379310344827587), None) -0.6048283081523658
521
0.1
((512, 1000, 2, 0.4666666666666667, 0.29351724137931035, 2.896551724137931, 1.1379310344827587), None) -0.720744961880223
522
0.1
((16, 2000, 6, 0.2888888888888889, 0.4139655172413793, 0.0, 0.9310344827586207), None) -1.3879239488935968
523
0.1
((16, 2000, 3, 0.37777777777777777, 0.19027586206896552, 1.8620689655172413, 0.6206896551724138), None) -1.416350976850779
524
0.1
((5

In [ ]:
for scores, actions in zip(scores_list_ori, actions_list_ori):
    print(actions[np.argmax(scores)][0], scores[np.argmax(scores)])

(335, 4.348633064005995) 0.03436411662463359
(375, 8.056192227792936) 0.03387046171330028
(267, 12.158172579216787) 0.03839516208255515
(412, 7.819539515210791) 0.03255006780027559
(449, 11.44821444147035) 0.033082805223584796
(365, 5.610780864444103) 0.032638915566141534
(426, 4.664170014115522) 0.0322694725026607
(480, 3.9542118763690866) 0.03496507206821923
(391, 8.68726612801199) 0.03557410140924732
(124, 9.239455790703662) 0.04142491925257696


our hybrid optimization method takes advantage of reinforcement learning to handle the discrete parameters, which successfully reduces the dimension of the parameter space for Bayesian Optimization. The technique improves the accuracy
of the solutions, compared with Bayesian Optimization and Random Search. The overhead of the one step Bayesian Optimization
is overcomed by highly reducing the iteration steps of Bayesian Optimization in the hybrid method because of the 
dimension reduction. The overall iteration steps is also compressed compared with Random Search to reach the global
minimizer.